# Scrape Estasy Product info : 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from  urllib.parse import urlparse, urljoin
url = "https://ecstasybd.com/?page=product-details&pid=6823"

HEADERS = {
    "Accept": "text/html",
    "Accept-Encoding": "UTF-8", 
    "User-Agent": "Mozilla/5.0"
    
}
response = requests.get(url, headers=HEADERS, timeout=5)
print(response.status_code)

200


In [2]:
response.headers

{'Server': 'nginx', 'Date': 'Thu, 12 Feb 2026 01:20:45 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '153260', 'Connection': 'keep-alive', 'X-Powered-By': 'PHP/7.4.33, PleskLin', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate', 'Pragma': 'no-cache', 'Set-Cookie': 'PHPSESSID=772o4asouof69dcrums8rgrvsk; path=/', 'X-Cache-Status': 'MISS', 'Strict-Transport-Security': 'max-age=15768000; includeSubDomains'}

In [3]:
response.content

b'<!DOCTYPE html>\n<html class="no-js" lang="zxx"><head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n\t<meta http-equiv="cache-control" content="no-cache" />\n    <meta name="description" content="From its humble beginnings as a single-store chain in 1997, Ecstasy has quickly become one of the nation\xe2\x80\x99s largest fashion retailers. Ecstasy lifestyle store offers exclusive and trendy merchandise. Where you can take your lifestyle to the limit and live life to the fullest.">\n\t <!--FACEBOOK-->\n\t\n\t<meta property="og:url" content="https://ecstasybd.com">\n\t<meta property="og:type" content="website">\n\t<meta property="og:title" content="Ecstasy BD">\n\t    <meta property="og:image" content="https://ecstasybd.com/ecstasybdMeta3.png?v=31997944">\n    <meta property="og:image:type" content="image/png">\n    <meta property="og:image:width" content="1024">\n    <me

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')
soup

<!DOCTYPE html>

<html class="no-js" lang="zxx"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="no-cache" http-equiv="cache-control">
<meta content="From its humble beginnings as a single-store chain in 1997, Ecstasy has quickly become one of the nation’s largest fashion retailers. Ecstasy lifestyle store offers exclusive and trendy merchandise. Where you can take your lifestyle to the limit and live life to the fullest." name="description"/>
<!--FACEBOOK-->
<meta content="https://ecstasybd.com" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="Ecstasy BD" property="og:title"/>
<meta content="https://ecstasybd.com/ecstasybdMeta3.png?v=31997944" property="og:image"/>
<meta content="image/png" property="og:image:type"/>
<meta content="1024" property="og:image:width"/>
<meta content="1024" property="og:image:height"/>
<meta content="From 

In [6]:
header_container = soup.header.find("div", class_ = "container").find("ul", class_ = "desktop-menu-ul")
header_container

<ul class="desktop-menu-ul">
<!--<li class=""><a href="?page=sub-category&mid=1" class="main-menu">MAN</a></li>-->
<li class="has-submenu"><a class="" href="?page=sub-category&amp;mid=1" id="tris-bottom-link-1">MAN</a>
<!-------The hover code was here-------->
<!--<li class=""><a href="?page=sub-category&mid=2" class="main-menu">WOMAN</a></li>-->
<li class="has-submenu"><a class="" href="?page=sub-category&amp;mid=2" id="tris-bottom-link-2">WOMAN</a>
<!-------The hover code was here-------->
<!--<li class=""><a href="?page=sub-category&mid=3" class="main-menu">LIFESTYLE</a></li>-->
<li class="has-submenu"><a class="" href="?page=sub-category&amp;mid=3" id="tris-bottom-link-3">LIFESTYLE</a>
<!-------The hover code was here-------->
<li class=""><a class="main-menu" href="?page=gift-voucher">GIFT VOUCHER</a></li>
</li></li></li></ul>

In [138]:
category_menu_li_tags = soup.header \
    .find("div", class_="container") \
    .find("ul", class_="desktop-menu-ul") \
    .find_all("li")
base_url = "https://ecstasybd.com/"

category_menu_item  = {}

for li in category_menu_li_tags:
    a_tag = li.find("a")
    if a_tag:
        item_name = a_tag.get_text(strip = True)
        if a_tag["href"].startswith("http"):
            item_url = a_tag["href"]
        else:
            item_url = base_url + a_tag["href"]
    category_menu_item[item_name] = item_url

category_menu_item

{'MAN': 'https://ecstasybd.com/?page=sub-category&mid=1',
 'WOMAN': 'https://ecstasybd.com/?page=sub-category&mid=2',
 'LIFESTYLE': 'https://ecstasybd.com/?page=sub-category&mid=3',
 'GIFT VOUCHER': 'https://ecstasybd.com/?page=gift-voucher'}

## Extract Subcategory url for each category
- Using selenium here for jabascript items showing

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get("https://ecstasybd.com/")
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="tris-bottom-link-1"]').click()
time.sleep(5)
html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

subcategory_item_type_li_tags = soup.body.header.find("div", class_='teal-bottom-header').ul.find_all("li",recursive = False)

subcategory_url = {}
base_url = "https://ecstasybd.com/"
for subcategory in subcategory_item_type_li_tags:

    parent_a_tag = subcategory.find("a", recursive=False)

    subcategory_name = parent_a_tag.get_text(strip=True)

    if 'http' not in parent_a_tag['href']:
        url = base_url + parent_a_tag['href']
    else:
        url = parent_a_tag['href']
    subcategory_url[subcategory_name] = url

subcategory_url

{'SS 26': 'https://ecstasybd.com/?page=product-list&sid=68',
 'Suits & Blazer': 'https://ecstasybd.com/?page=product-list&sid=72',
 'T-Shirt': 'https://ecstasybd.com/?page=product-list&sid=19',
 'Shirt': 'https://ecstasybd.com/?page=product-list&sid=4',
 'Polo': 'https://ecstasybd.com/?page=product-list&sid=3',
 'Bottom': 'https://ecstasybd.com/?page=product-list&sid=5',
 'Ethnic Wear': 'https://ecstasybd.com/?page=product-list&sid=18',
 'Winterwear': 'https://ecstasybd.com/?page=product-list&sid=70',
 'Underwear': 'https://ecstasybd.com/?page=product-list&sid=57',
 'Footwear': 'https://ecstasybd.com/?page=product-list&sid=6',
 'Accessories': 'https://ecstasybd.com/?page=product-list&sid=42',
 'Tanjim Squad': 'https://ecstasybd.com/?page=product-list&sid=2'}

In [ ]:
category_subcategory_li_tags = soup.header.find("div", class_ = "teal-bottom-header").ul.find_all("li", recursive = False)

result = {}
for li in category_subcategory_li_tags:
    a = li.find("a", recursive=False)
    if not a:
        continue
    category_name = a.get_text(strip=True)
    if li.find("div"):
        sub_lis = li.find_all("li")
        
        sub_category_name = [item.text for item in li.find_all("li")]
        result[category_name] = sub_category_name
    else:
        result[category_name] = [None]

result

{'SS 26': [None],
 'Suits & Blazer': [None],
 'T-Shirt': [None],
 'Shirt': ['Formal',
  'Casual',
  'Half Sleeve',
  'Full Sleeve',
  'Printed',
  'Solid',
  'Club'],
 'Polo': [None],
 'Bottom': ['Jeans',
  'Chinos',
  'Formal',
  'Joggers',
  'Cargo',
  'Shorts',
  'Pajama'],
 'Ethnic Wear': ['Panjabi', 'Kabli', 'Vest'],
 'Winterwear ': ['Winter Shirt', 'Jacket', 'Sweater', 'Sweatshirt', 'Hoodie'],
 'Underwear': [None],
 'Footwear': ['Sneakers', 'Sandal', 'Boot', 'Loafer'],
 'Accessories': ['Mask', 'Tie', 'Belt'],
 'Tanjim Squad': ['T-Shirt ',
  'Shirt',
  'Joggers',
  'Shorts',
  'Tanjim Squad Sneakers']}

In [49]:
category_subcategory_items = {}

for li in category_subcategory_li_tags:
    a = li.find("a", recursive=False)
    if not a:
        continue

    category_name = a.get_text(strip=True)

    # FIX: search inside li
    sub_ul = li.find("ul")

    if sub_ul:
        sub_categories_name = [
            sub_li.get_text(strip=True)
            for sub_li in sub_ul.find_all("li", recursive=False)
        ]
        category_subcategory_items[category_name] = sub_categories_name
    else:
        category_subcategory_items[category_name] = []

category_subcategory_items


{'SS 26': [],
 'Suits & Blazer': [],
 'T-Shirt': [],
 'Shirt': ['Formal',
  'Casual',
  'Half Sleeve',
  'Full Sleeve',
  'Printed',
  'Solid',
  'Club'],
 'Polo': [],
 'Bottom': ['Jeans',
  'Chinos',
  'Formal',
  'Joggers',
  'Cargo',
  'Shorts',
  'Pajama'],
 'Ethnic Wear': ['Panjabi', 'Kabli', 'Vest'],
 'Winterwear': ['Winter Shirt', 'Jacket', 'Sweater', 'Sweatshirt', 'Hoodie'],
 'Underwear': [],
 'Footwear': ['Sneakers', 'Sandal', 'Boot', 'Loafer'],
 'Accessories': ['Mask', 'Tie', 'Belt'],
 'Tanjim Squad': ['T-Shirt',
  'Shirt',
  'Joggers',
  'Shorts',
  'Tanjim Squad Sneakers']}

In [72]:
extend_category_subcategory_items = []
for key, values in category_subcategory_items.items():
    if len(values) <= 0:
        extend_category_subcategory_items.append((key, None))
    else:
        for value in values:
            extend_category_subcategory_items.append((key, value))
        

    
extend_category_subcategory_items

[('SS 26', None),
 ('Suits & Blazer', None),
 ('T-Shirt', None),
 ('Shirt', 'Formal'),
 ('Shirt', 'Casual'),
 ('Shirt', 'Half Sleeve'),
 ('Shirt', 'Full Sleeve'),
 ('Shirt', 'Printed'),
 ('Shirt', 'Solid'),
 ('Shirt', 'Club'),
 ('Polo', None),
 ('Bottom', 'Jeans'),
 ('Bottom', 'Chinos'),
 ('Bottom', 'Formal'),
 ('Bottom', 'Joggers'),
 ('Bottom', 'Cargo'),
 ('Bottom', 'Shorts'),
 ('Bottom', 'Pajama'),
 ('Ethnic Wear', 'Panjabi'),
 ('Ethnic Wear', 'Kabli'),
 ('Ethnic Wear', 'Vest'),
 ('Winterwear', 'Winter Shirt'),
 ('Winterwear', 'Jacket'),
 ('Winterwear', 'Sweater'),
 ('Winterwear', 'Sweatshirt'),
 ('Winterwear', 'Hoodie'),
 ('Underwear', None),
 ('Footwear', 'Sneakers'),
 ('Footwear', 'Sandal'),
 ('Footwear', 'Boot'),
 ('Footwear', 'Loafer'),
 ('Accessories', 'Mask'),
 ('Accessories', 'Tie'),
 ('Accessories', 'Belt'),
 ('Tanjim Squad', 'T-Shirt'),
 ('Tanjim Squad', 'Shirt'),
 ('Tanjim Squad', 'Joggers'),
 ('Tanjim Squad', 'Shorts'),
 ('Tanjim Squad', 'Tanjim Squad Sneakers')]

In [73]:
import pandas as pd
pd.DataFrame(extend_category_subcategory_items, columns=["category","sub_category" ])

,category,sub_category
0,SS 26,None
1,Suits & Blazer,None
2,T-Shirt,None
3,Shirt,Formal
4,Shirt,Casual
5,Shirt,Half Sleeve
6,Shirt,Full Sleeve
7,Shirt,Printed
8,Shirt,Solid
9,Shirt,Club


## Extract Main product Details

# Estasy Product Info Extract

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from  urllib.parse import urlparse, urljoin
url = "https://ecstasybd.com/?page=product-details&pid=5463"

HEADERS = {
    "Accept": "text/html",
    "Accept-Encoding": "UTF-8", 
    "User-Agent": "Mozilla/5.0"
    
}
response = requests.get(url, headers=HEADERS, timeout=5)
print(response.status_code)

soup = BeautifulSoup(response.text, 'html.parser')

product_detail_page = soup.body.main.find("div", class_="shop-main-wrapper section-space product-details-page")
product_detail_page

product_quick_details = product_detail_page.find("div", class_ = "product-details-des quick-details")

related_products = soup.body.main.find("section", class_ = "related-products").find_all("a", href = lambda x: "pid" in x)


## Product Images extraction
base_url = "https://ecstasybd.com/"

product_images_links = []

for link in product_detail_page.find_all("div", class_="pro-large-img img-zoom"):
    img_tag = link.find("img")
    
    if not img_tag or not img_tag.get("src"):
        continue
        
    img = img_tag["src"]
    
    if not img.startswith("http"):
        img = base_url + img
        
    product_images_links.append(img)

product_images_links

product_details = {
    "product-name": product_quick_details.find("h3", class_ = "product-name").get_text(strip=True),

    "price-regular" : product_quick_details.find("span", class_="price-regular price-details").get_text(strip=True),

    "sizes" : product_quick_details.table.find_all("td")[1].get_text(strip=True),

    "availability" : product_quick_details.find("span", class_ = "product-att availability").get_text(strip=True),

    "SKU" : product_quick_details.find("div", class_="sku-wrap half-wrap").find("span", class_ = "product-att sku").get_text(strip=True),

    "PID" : product_quick_details.find("div", class_="pid-container").find("span", class_ = "product-att sku").get_text(strip=True),

    "related-PID" : list(set([pid['href'].split("pid=")[1] for pid in related_products])),
    
    "info_guide" : product_quick_details.find("div", class_="product-review-info").find("p").get_text(),

    "product-images-links": product_images_links

}

product_details

200


{'product-name': 'CASUAL SHIRT',
 'price-regular': 'TK. 2486',
 'sizes': 'L, XL, 2XL ,',
 'availability': 'In Stock',
 'SKU': '10003663',
 'PID': 'PI5463',
 'related-PID': ['5095'],
 'info_guide': '\nMaterial: 100% Cotton CorduroyDesign: Features two functional chest pockets, and a classic collar.Fit: Regular fit for comfortable, everyday wear.Model usually wears size L top and 32 waist bottoms.Model is wearing size LFits true to sizeHeight: 6’2” (183 cm)Chest: 38” (97 cm)Shoulder: 18.5” (47 cm)\n',
 'product-images-links': ['https://ecstasybd.com/all-images/product/Product-Image-1731841846.jpg',
  'https://ecstasybd.com/all-images/product/Product-Image-11731841846.jpg',
  'https://ecstasybd.com/all-images/product/Product-Image-21731841846.jpg',
  'https://ecstasybd.com/all-images/product/Product-Image-31731922708.png']}

In [129]:
product_details

{'product-name': 'TANJIM POLO',
 'price-regular': 'TK. 3267',
 'sizes': 'L,',
 'availability': 'In Stock',
 'SKU': '10003958',
 'PID': 'PI6330',
 'related-PID': ['6133'],
 'info_guide': '\nLight brown/tan polo shirt\n\n\nBlack collar and sleeve cuffs with brown/beige stripes\n\n\nFabric:\xa095% cotton, 5% spnadex\n\n\nThree-button placket\n\n\nSmall square logo on left chest\n\n\nRegular fit, slightly tailored around arms and chest\n\n\nCasual to smart-casual style\n',
 'product-images-links': ['https://ecstasybd.com/all-images/product/Product-Image-1754654999.jpg',
  'https://ecstasybd.com/all-images/product/Product-Image-11754654999.jpg',
  'https://ecstasybd.com/all-images/product/Product-Image-21754654964.jpg',
  'https://ecstasybd.com/all-images/product/Product-Image-31754655839.jpg']}